In [17]:
from pathlib import Path
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from bs4 import BeautifulSoup
import requests
import csv
from datetime import datetime

In [18]:
extentions = [".pem"]
root_path = '/etc/ssl/certs'

In [19]:
blacklist_cert_list=[]

In [20]:
def loadBlacklistedSSls():
    CSV_URL="https://sslbl.abuse.ch/blacklist/sslblacklist.csv"

    download = requests.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    lines = decoded_content.splitlines()
    cr = csv.reader(lines, delimiter=',')
    my_list = list(cr)
    headers = None
    results = []
    for row in my_list:
        if len(row) == 3:
            if headers is None:
                headers = row
                headers[0]=headers[0].strip('#').strip()
                continue
            results.append(dict(zip(headers,row)))
    return results
blacklist_cert_list += loadBlacklistedSSls()
blacklist_cert_list[-1]

{'Listingdate': '2014-05-04 08:09:56',
 'SHA1': 'b08a4939fb88f375a2757eaddc47b1fb8b554439',
 'Listingreason': 'Shylock C&C'}

In [21]:
print("Please review the list and make sure you do not have any banned certs")
for item in Path(root_path).iterdir():
    if item.is_file():
        if item.suffix in extentions:
            cert = x509.load_pem_x509_certificate(item.read_bytes(), default_backend())
            sha1_hashbytes = cert.fingerprint(hashes.SHA1())
            sha1_spaces= " ".join([format(i,'02x') for i in sha1_hashbytes])
            sha1 = "".join([format(i,'02x') for i in sha1_hashbytes])
            sha256_hashbytes = cert.fingerprint(hashes.SHA256())
            sha256 = " ".join([format(i,'02x') for i in sha256_hashbytes])
            trusted = False
            for b in blacklist_cert_list:
                if 'SHA1' in b and b['SHA1'] == sha1:
                    print
                    print("###### BANNED CERT:",b['Listingreason']) 
                    print('Filename:',item.stem)
                    print('Issuer:',cert.issuer)
                    print('Subject:',cert.subject)
                    print('Expires After:',cert.not_valid_after)
                    print('SN:',cert.serial_number)
                    print(sha1)
                    print(sha256)
print('>finished')

Please review the list and make sure you do not have any banned certs
>finished


In [22]:
trusted_ca_list = []

In [23]:
def loadVisaTrustedCAs():
    url = "https://developer.visa.com/pages/trusted_certifying_authorities"
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    table = soup.find('table')
    if table is None:
        raise Exception(f"Table missing please check: {url}")
    first = True
    results = []
    for row in table.find_all('tr'):
        if first:
            first = False
            continue
        tds = row.find_all('td')
        results.append({
            'CA':tds[0].text.strip(),
            'Root Certificate Types':tds[1].text.strip(),
            'SHA1-Thumbprint':tds[2].text.strip().strip('\u200e'),
            'src':"Visa"
        })
    return results
trusted_ca_list += loadVisaTrustedCAs()
trusted_ca_list[-1]

{'CA': 'Verisign',
 'Root Certificate Types': 'VeriSign Universal Root Certification Authority',
 'SHA1-Thumbprint': '36 79 ca 35 66 87 72 30 4d 30 a5 fb 87 3b 0f a7 7b b7 0d 54',
 'src': 'Visa'}

In [24]:
def loadMicrosoftTrustedCAs():
    CSV_URL="https://ccadb-public.secure.force.com/microsoft/IncludedCACertificateReportForMSFTCSV"

    download = requests.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    lines = decoded_content.splitlines()
    cr = csv.reader(lines, delimiter=',')
    my_list = list(cr)
    headers = None
    results = []
    for row in my_list:
        
        if len(row) == 10:
            if headers is None:
                headers = row
                headers[0]=headers[0].strip('#').strip()
                continue
            data = dict(zip(headers,row))
            data['src']='Microsoft'
            results.append(data)
    return results
trusted_ca_list += loadMicrosoftTrustedCAs()
trusted_ca_list[-1]

{'CA Owner': 'Zetes',
 'CA Common Name or Certificate Name': 'ZETES TSP ROOT CA 001',
 'SHA-1 Fingerprint': '3753D295FC6D8BC39B375650BFFC821AED504E1A',
 'SHA-256 Fingerprint': '016E1DCD5F78841BBEBBAE9DDEA08C8D7EC54E698E95BB778ECDD1E10D8BF4F9',
 'Valid From [GMT]': '2016 May 20',
 'Valid To [GMT]': '2036 May 20',
 'Public Key Algorithm': 'RSA 4096 bits',
 'Signature Hash Algorithm': 'SHA256WithRSA',
 'Microsoft EKUs': 'Client Authentication;Document Signing;Secure Email;Time Stamping',
 'Microsoft Status': 'Included',
 'src': 'Microsoft'}

In [25]:
def loadAppleTrustedCAs():
    url="https://support.apple.com/en-us/HT212773#trusted"
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    table = soup.find('table')
    if table is None:
        raise Exception(f"Table missing please check: {url}")
    first = True
    results = []
    for row in table.find_all('tr'):
        if first:
            first = False
            continue
        tds = row.find_all('td')
        results.append({
            'CA':tds[0].text.strip(),
            'Root Certificate Types':tds[2].text.strip(),
            'SHA256-Thumbprint':tds[8].text.strip(),
            "SRC":"Apple"
        })
    return results
trusted_ca_list += loadAppleTrustedCAs()

In [26]:
print('Certs listed below are expired')
print()
today = datetime.now()
for item in Path(root_path).iterdir():
    if item.is_file():
        if item.suffix in extentions:
            cert = x509.load_pem_x509_certificate(item.read_bytes(), default_backend())
            
            if(cert.not_valid_after<today):
                print('Expired Cert', cert)
                print('Filename:',item.stem)
                print('Issuer:',cert.issuer)
                print('Subject:',cert.subject)
                print('Expires After:',cert.not_valid_after)
                print('SN:',cert.serial_number)
                print(sha1)
                print(sha256)
                print()
print('>finished')

Certs listed below are expired

Expired Cert <Certificate(subject=<Name(OU=GlobalSign Root CA - R2,O=GlobalSign,CN=GlobalSign)>, ...)>
Filename: GlobalSign_Root_CA_-_R2
Issuer: <Name(OU=GlobalSign Root CA - R2,O=GlobalSign,CN=GlobalSign)>
Subject: <Name(OU=GlobalSign Root CA - R2,O=GlobalSign,CN=GlobalSign)>
Expires After: 2021-12-15 08:00:00
SN: 4835703278459682885658125
0d44dd8c3c8c1a1a58756481e90f2e2affb3d26e
18 ce 6c fe 7b f1 4e 60 b2 e3 47 b8 df e8 68 cb 31 d0 2e bb 3a da 27 15 69 f5 03 43 b4 6d b3 a4

Expired Cert <Certificate(subject=<Name(O=Cybertrust\, Inc,CN=Cybertrust Global Root)>, ...)>
Filename: Cybertrust_Global_Root
Issuer: <Name(O=Cybertrust\, Inc,CN=Cybertrust Global Root)>
Subject: <Name(O=Cybertrust\, Inc,CN=Cybertrust Global Root)>
Expires After: 2021-12-15 08:00:00
SN: 4835703278459682877484360
0d44dd8c3c8c1a1a58756481e90f2e2affb3d26e
18 ce 6c fe 7b f1 4e 60 b2 e3 47 b8 df e8 68 cb 31 d0 2e bb 3a da 27 15 69 f5 03 43 b4 6d b3 a4

>finished


In [28]:
print('Please verify any certs listed below')
print()
for item in Path(root_path).iterdir():
    if item.is_file():
        if item.suffix in extentions:
            cert = x509.load_pem_x509_certificate(item.read_bytes(), default_backend())
            sha1_hashbytes = cert.fingerprint(hashes.SHA1())
            sha1_spaces= " ".join([format(i,'02x') for i in sha1_hashbytes])
            sha1 = "".join([format(i,'02x') for i in sha1_hashbytes])
            sha256_hashbytes = cert.fingerprint(hashes.SHA256())
            sha256_spaces = " ".join([format(i,'02x') for i in sha256_hashbytes])
            sha256 = "".join([format(i,'02x') for i in sha256_hashbytes])
            trusted = False
            for t in trusted_ca_list:
                if'SHA1-Thumbprint' in t:
                    if t['SHA1-Thumbprint'] == sha1_spaces:
                        trusted = True
                        break
                    else:
                        issue=f"SHA1-Thumbprint {sha1_spaces} != {t['SHA1-Thumbprint']}"
                        break
                        
                if'SHA256-Thumbprint' in t:
                    if t['SHA256-Thumbprint'] == sha256_spaces:
                        trusted = True
                        break
                    else:
                        issue=f"SHA256-Thumbprint {sha256_spaces} != {t['SHA256-Thumbprint']}"
                        break
                        
                if 'SHA-1 Fingerprint' in t:
                    if t['SHA-1 Fingerprint'] == sha1.upper():
                        trusted = True
                        break
                    else:
                        issue=f"SHA-1 Fingerprint {sha1.upper()} != {t['SHA-1 Fingerprint']}"
                        break
                        
                if 'SHA-256 Fingerprint' in t:
                    if t['SHA-256 Fingerprint'] == sha256.upper():
                        trusted = True
                        break
                    else:
                        issue=f"SHA-256 Fingerprint {sha256.upper()} != {t['SHA-256 Fingerprint']}"
                        break
                        
                
            if trusted == False:
                print('Filename:',item.stem)
                print('Issuer:',cert.issuer)
                print('Subject:',cert.subject)
                print('Expires After:',cert.not_valid_after)
                print('SN:',cert.serial_number)
                print('Issue:',issue)
                print("Trust By", t['src'])
                print(sha1)
                print(sha256)
                print()
print('>finished')

Please verify any certs listed below

Filename: QuoVadis_Root_CA_3
Issuer: <Name(C=BM,O=QuoVadis Limited,CN=QuoVadis Root CA 3)>
Subject: <Name(C=BM,O=QuoVadis Limited,CN=QuoVadis Root CA 3)>
Expires After: 2031-11-24 19:06:44
SN: 1478
Issue: SHA1-Thumbprint 1f 49 14 f7 d8 74 95 1d dd ae 02 c0 be fd 3a 2d 82 75 51 85 != f3 73 b3 87 06 5a 28 84 8a f2 f3 4a ce 19 2b dd c7 8e 9c ac
Trust By Visa
1f4914f7d874951dddae02c0befd3a2d82755185
18f1fc7f205df8adddeb7fe007dd57e3af375a9c4d8d73546bf4f1fed1e18d35

Filename: Certigna
Issuer: <Name(C=FR,O=Dhimyotis,CN=Certigna)>
Subject: <Name(C=FR,O=Dhimyotis,CN=Certigna)>
Expires After: 2027-06-29 15:13:05
SN: 18364802974209362175
Issue: SHA1-Thumbprint b1 2e 13 63 45 86 a4 6f 1a b2 60 68 37 58 2d c4 ac fd 94 97 != f3 73 b3 87 06 5a 28 84 8a f2 f3 4a ce 19 2b dd c7 8e 9c ac
Trust By Visa
b12e13634586a46f1ab2606837582dc4acfd9497
e3b6a2db2ed7ce48842f7ac53241c7b71d54144bfb40c11f3f1d0b42f5eea12d

Filename: DigiCert_Assured_ID_Root_G2
Issuer: <Name(C=US,O=D